In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from skimage.util import view_as_windows
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier
from tqdm.notebook import tqdm

sp_key = ('m00', 'm10', 'm01', 'm20', 'm11', 'm02', 'm30', 'm21', 'm12', 'm03')
nu_key = ('nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12', 'nu03')

ch_avg = [f'ch_avg_{x}' for x in 'BGR']
ch_std = [f'ch_std_{x}' for x in 'BGR']
hu_mom = [f'hu_mom{x + 1}' for x in range(7)]
sp_mom = [f'sp_mom_{x[1:]}' for x in sp_key]
nu_mom = [f'nu_mom_{x[2:]}' for x in nu_key]

features = ch_avg + ch_std + hu_mom + sp_mom + nu_mom


def get_metrics(result: np.ndarray, real: np.ndarray):
    conf = confusion_matrix(result.flatten(), real.flatten())
    TN, FP, FN, TP = conf.ravel()

    accuracy = 1.0 * (TP + TN) / (TP + TN + FP + FN)
    sensitivity = 1.0 * TP / (TP + FN)
    specificity = 1.0 * TN / (TN + FP)

    return accuracy, sensitivity, specificity


def get_data(path='./drive/MyDrive/HRF/'):
    files = sorted(os.listdir(path))
    data = {
        'original': [],
        'labeled': [],
        'mask': []
    }
    dim = (876, 584)
    fun = lambda src: cv2.resize(cv2.imread(src), dim, interpolation = cv2.INTER_AREA)
    fun_with_0 = lambda src: cv2.resize(cv2.imread(src, 0), dim, interpolation = cv2.INTER_AREA)
    for file in files:
        if file.endswith('h.jpg'):
            data['original'].append(fun(src=f'{path}{file}'))
        elif file.endswith('h.tif'):
            data['labeled'].append(fun_with_0(src=f'{path}{file}'))
        elif file.endswith('h_mask.tif'):
            data['mask'].append(fun_with_0(src=f'{path}{file}'))
    return data


def split_image(image, tile_size=5, step=None):
    gray = (len(image.shape) == 2) or (image.shape[2] == 1)
    if not step: step = tile_size
    return view_as_windows(
        image, (tile_size, tile_size), step=step).reshape(-1, tile_size, tile_size) if gray else view_as_windows(
        image, (tile_size, tile_size, 3), step=step).reshape(-1, tile_size, tile_size, 3)


def get_features(image):
    channels = image.reshape(-1, 3)
    mean, std = np.mean(channels, axis=0), np.std(channels, axis=0)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mom = cv2.moments(gray)
    hu = cv2.HuMoments(mom)[:, 0]
    sp, nu = [mom[k] for k in sp_key], [mom[k] for k in nu_key]
    return np.hstack([mean, std, hu, sp, nu])


def create_meta_data_row(row, step=None):
    image, label = row['original'], row['labeled']
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    image = cv2.bilateralFilter(image, 5, 13, 13)
    image[:,:,0] = cv2.createCLAHE(clipLimit=3).apply(image[:,:,0])
    image = cv2.cvtColor(image, cv2.COLOR_LAB2BGR)
    X = np.array([get_features(x) for x in split_image(image, step=step)])
    y = split_image(label, step=step)[:, 3, 3] > 0
    y = y.astype(int)
    return X, y


def meta_data(data):
    total = pd.DataFrame()
    for _, row in tqdm(list(data.iterrows())):
        X, y = create_meta_data_row(row, step=1)
        chunk = pd.DataFrame(data=X, columns=features)
        chunk['id'] = row.id
        chunk['label'] = y
        total = pd.concat([total, chunk])
    return total.reset_index(drop=True)


def create_data():
    data = pd.DataFrame(get_data()).loc[:, ['original', 'labeled']]
    data['id'] = data.index
    return meta_data(data[5:])

In [ ]:
train = create_data()

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
test = create_data()

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizer_v2.adam import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy

In [ ]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(30,)))
model.add(Dense(50, activation="relu"))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=1e-3), loss=binary_crossentropy,
                metrics=['accuracy'])

keras_callbacks = [
    ModelCheckpoint('./output/NN_model', save_weights_only=True, monitor='val_loss', mode='min', save_best_only=True),
    EarlyStopping(monitor="val_loss", mode="min", patience=50, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", mode="min", patience=10, verbose=1),
  ]

In [ ]:
epochs = 100
batch = 128

X_train, y_train = train.drop(['id', 'label'], axis=1), train['label']
sampler = RandomUnderSampler()
X_train, y_train = sampler.fit_resample(X_train, y_train)
X_test, y_test = test.drop(['id', 'label'], axis=1), test['label']
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
history = model.fit(
    X_train, y_train,
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=keras_callbacks
)

Epoch 1/100
7816/7816 [==============================] - 27s 3ms/step - loss: 7.6258 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 2/100
7816/7816 [==============================] - 25s 3ms/step - loss: 7.6257 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 3/100
7816/7816 [==============================] - 25s 3ms/step - loss: 7.6258 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 4/100
7816/7816 [==============================] - 25s 3ms/step - loss: 7.6258 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 5/100
7816/7816 [==============================] - 23s 3ms/step - loss: 7.6258 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 6/100
7816/7816 [==============================] - 25s 3ms/step - loss: 7.6258 - accuracy: 0.4999 - val_loss: 7.6197 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 7/100
7816/7816 [=======================

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

model.save_weights('./output/NN_model.h5')

[13.232207298278809, 0.1321563571691513]


In [ ]:
model